In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl

In [2]:

results_dir = Path("/nfs/chisholmlab001/kve/2021_dark_adapted_transcriptome/results_bwa_samse_filtered/")

experiment_dir = results_dir / "experiments"
plotting_dir = results_dir / 'gene_expression_plots'

log_count_table = experiment_dir / "experiment_all" / "DEseq2out" / "experiment_all_NATL2A_rlog.tsv"
rlog_df = pd.read_csv(log_count_table, sep='\t', index_col="long_ID")

rlog_df.columns = pd.MultiIndex.from_product([['Control', 'Pheno'], [0, 4, 8, 13, 16, 20, 24], [1, 2, 3]], names=['treatment', 'time', 'replicate'])

rlog_mean_df = rlog_df.groupby(level=['treatment', 'time'], axis='columns').mean()

rlog_std_df = rlog_df.groupby(level=['treatment', 'time'], axis='columns').std()

rlog_df

treatment           Control                                            \
time                     0                               4              
replicate                 1          2          3         1         2   
long_ID                                                                 
cds-PMN2A_RS00335  4.382073   4.769943   4.437839  5.635738  6.001895   
cds-PMN2A_RS01135  8.514615   8.179944   8.111122  7.874838  7.785570   
cds-PMN2A_RS05260  6.530092   6.805233   6.923710  7.469956  7.396946   
cds-PMN2A_RS09840  9.975688  10.103570  10.139084  9.344042  9.477225   
cds-PMN2A_RS09995  6.772258   6.877898   6.980190  7.081970  6.866691   
...                     ...        ...        ...       ...       ...   
rna-PMN2A_RS08985  5.640378   5.928088   5.636179  5.826790  5.498603   
rna-PMN2A_RS09060  6.424294   6.473093   6.313313  5.419050  5.449206   
rna-PMN2A_RS09160  7.201996   6.983364   6.981437  7.761996  7.860478   
rna-PMN2A_RS09485  9.866696   9.701866   9.542331  9.108499  9.487813   
rna-PMN2A_RS09710  9.368325   8.897513   8.633345  8.735696  9.313096   

treatment                                                             ...  \
time                                8                             13  ...   
replicate                 3          1         2         3         1  ...   
long_ID                                                               ...   
cds-PMN2A_RS00335  5.315157   5.321684  5.270754  5.348773  6.260072  ...   
cds-PMN2A_RS01135  7.875105   8.895097  8.409313  8.465855  8.721563  ...   
cds-PMN2A_RS05260  7.200095   5.693572  6.256993  6.173637  6.593834  ...   
cds-PMN2A_RS09840  9.220029   9.177195  9.079928  8.919935  9.569534  ...   
cds-PMN2A_RS09995  7.031456   5.914439  6.480779  6.794663  7.061649  ...   
...                     ...        ...       ...       ...       ...  ...   
rna-PMN2A_RS08985  5.926936   5.912726  5.359425  5.239483  6.115551  ...   
rna-PMN2A_RS09060  6.100896   6.203453  5.346275  6.074755  4.558018  ...   
rna-PMN2A_RS09160  8.639304   9.543887  8.746406  8.658636  6.624920  ...   
rna-PMN2A_RS09485  9.747332   9.309517  8.315968  8.903824  8.227736  ...   
rna-PMN2A_RS09710  9.229782  10.143721  8.334089  9.069988  6.948525  ...   

treatment             Pheno                                           \
time                     13        16                             20   
replicate                 3         1         2         3          1   
long_ID                                                                
cds-PMN2A_RS00335  5.513053  5.502516  5.205224  5.288963   4.640463   
cds-PMN2A_RS01135  8.500619  8.275963  8.302217  8.391940   8.086343   
cds-PMN2A_RS05260  6.649573  6.012024  6.398379  6.518047   5.924271   
cds-PMN2A_RS09840  9.790996  9.642471  9.837612  9.552368  10.264614   
cds-PMN2A_RS09995  7.171745  7.250824  7.116179  7.228078   7.214901   
...                     ...       ...       ...       ...        ...   
rna-PMN2A_RS08985  6.421996  6.905512  6.477142  6.999735   6.632128   
rna-PMN2A_RS09060  3.948397  4.945524  4.752957  5.311388   5.931634   
rna-PMN2A_RS09160  6.643486  7.220839  7.033836  7.132445   7.264252   
rna-PMN2A_RS09485  8.606886  8.508518  8.479483  8.827619   9.198054   
rna-PMN2A_RS09710  6.831331  7.558818  7.124382  7.574929   8.284526   

treatment                                                                 
time                                            24                        
replicate                  2          3          1          2          3  
long_ID                                                                   
cds-PMN2A_RS00335   5.347424   4.926281   5.191271   5.618934   5.013509  
cds-PMN2A_RS01135   8.260859   7.924661   8.076918   8.818792   8.423162  
cds-PMN2A_RS05260   6.353475   6.077451   6.109280   5.998971   6.375528  
cds-PMN2A_RS09840  10.153374  10.426333  10.077871  10.153497  10.095950  
cds-PMN2A_RS09995   7.407044   7.473971   7.120275   7.291306   7.4

In [14]:
dfs = []
for time in [0, 4, 8, 13, 16, 20, 24]:
    result_path = experiment_dir / f"experiment_{time}" / "DGE_tables" / f"experiment_{time}_NATL2A_DGE_all.tsv"
    result_df = pd.read_csv(result_path, sep="\t", index_col="long_ID")

    annotation_data = result_df[["product", "protein_id", "locus_tag"]]

    significance_data = result_df[["log2FoldChange", "padj"]]
    significance_data.columns = pd.MultiIndex.from_product([[time], significance_data.columns], names=['time', 'significance'])
    dfs.append(significance_data)

result_df = pd.concat(dfs, axis=1)

result_df = result_df.swaplevel("time", "significance", axis=1)

conversion_df = pd.read_csv("gene_label_conversion_table.tsv", sep='\t', index_col='gene')

annotation_data = annotation_data.join(conversion_df)


In [15]:
annotation_data

,product,protein_id,locus_tag,pmGene,oldGeneName
long_ID,,,,,
cds-WP_011294510.1,form I ribulose bisphosphate carboxylase large...,WP_011294510.1,PMN2A_RS02860,PMN2A_RS02860,PMN2A_1879
cds-WP_011130577.1,BMC domain-containing protein,WP_011130577.1,PMN2A_RS02855,PMN2A_RS02855,PMN2A_1878
cds-WP_011294511.1,ribulose bisphosphate carboxylase small subunit,WP_011294511.1,PMN2A_RS02865,PMN2A_RS02865,PMN2A_1880
cds-WP_011295330.1,ATP synthase subunit b',WP_011295330.1,PMN2A_RS08155,PMN2A_RS08155,PMN2A_0984
cds-WP_011293983.1,type I glyceraldehyde-3-phosphate dehydrogenase,WP_011293983.1,PMN2A_RS00125,PMN2A_RS00125,PMN2A_1350
...,...,...,...,...,...
cds-WP_011294605.1-3,high light inducible protein,WP_011294605.1,PMN2A_RS05980,PMN2A_RS05980,NaN
cds-WP_011294648.1,high light inducible protein,WP_011294648.1,PMN2A_RS05845,PMN2A_RS05845,PMN2A_0552
cds-WP_011294648.1-2,high light inducible protein,WP_011294648.1,PMN2A_RS05985,PMN2A_RS05985,PMN2A_0577


In [11]:
@mpl.rc_context({
    'lines.linewidth': 6, 
    'lines.marker':'o', 
    'lines.markersize':18, 
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large'})
def plot_gene(ax, gene_ID, rlog_mean_df, rlog_std_df, results_df, annotation_data, night_periods, night_color, attr_dict):

    ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    mean_series = rlog_mean_df.loc[gene_ID]
    std_series = rlog_std_df.loc[gene_ID]

    for treatment in ['Control', 'Pheno']:
        for (s, e) in night_periods:
            ax.axvspan(s, e, color=night_color)
        ax.errorbar(x=mean_series[treatment].index, y=mean_series[treatment], yerr=std_series[treatment], capsize=6, capthick=3, label=treatment, color=attr_dict[treatment]['color'])
    
    # axes limits
    bottom, top = ax.get_ylim()
    y_range = top - bottom
    ax.set_ylim(top-(y_range*1.1), top)
    ax.set_xlim(-1, 25)
    
    #plotting significance
    significance_df = result_df.loc[gene_ID]
    significance_df["padj"]

    for x, padj in zip(significance_df["padj"].index, significance_df["padj"]):
        if padj < 0.05:
            ax.plot(x, bottom - y_range*0.05, color='k', marker=(8, 2, 0), markersize=15, label="Differentially expressed at 5% FDR")

    gene_annotation = annotation_data.loc[gene_ID]

    # title stuff
    ax.set_title(f"{gene_annotation.name}/{gene_annotation['locus_tag']}\n{gene_annotation['product']}")

@mpl.rc_context({
    'lines.linewidth': 6, 
    'lines.marker':'o', 
    'lines.markersize':18, 
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'xx-large',
    'axes.titlesize':'xx-large',
    'xtick.labelsize':'xx-large',
    'figure.titlesize': 'xx-large',
    'ytick.labelsize':'xx-large'})
def plot_gene_table(gene_df_subset, out_path, rlog_mean_df, rlog_std_df, results_df, annotation_data,
                num_cols = 3,
                night_periods = [(-11, 0), (13, 24)], 
                night_color="#dfdfdf",
                attr_dict={'Control':{'color':'salmon', 'label':'Parental $\it{Prochlorococcus}$'}, 'Pheno':{'color':'lightseagreen', 'label':'Dark-tolerant $\it{Prochlorococcus}$'}}):

    # original from elaina:
    # color_dict={'Control':'#e97e72', 'Pheno':'#52bcc2'}

    gene_arr = list(gene_df_subset.index.values)
    gene_arr += [None]*(num_cols - (len(gene_arr) % num_cols))
    gene_arr = np.array(gene_arr).reshape(-1, num_cols)

    y_height = 5
    x_width = 5

    heights = [y_height]*gene_arr.shape[0]
    widths = [x_width]*gene_arr.shape[1]

    fig = plt.figure(figsize=(sum(widths), sum(heights)), constrained_layout=True)
    gs = fig.add_gridspec(ncols=len(widths), nrows=len(heights), height_ratios=heights, width_ratios=widths)

    for i, row in enumerate(gene_arr):
        for j, element in enumerate(row):
            if element != None:
                ax = fig.add_subplot(gs[i,j])
                plot_gene(ax, element, rlog_mean_df, rlog_std_df, results_df, annotation_data, night_periods, night_color, attr_dict)

    # handles, labels = ax.get_legend_handles_labels()
    legend_elements = [mpl.lines.Line2D([0], [0], color=d['color'], label=d['label']) for t, d in attr_dict.items()]
    legend = fig.legend(handles=legend_elements, loc='center left', bbox_to_anchor= (1.01, 0.5))

    xlab = fig.supxlabel("Time (hours)")
    ylab = fig.supylabel("Relative transcript abundance")
    plt.savefig(out_path, bbox_extra_artists=[legend, xlab, ylab], bbox_inches='tight')
    plt.close()



In [ ]:
# Plot Clock Proteins
sigma_factors = ["PMN2A_RS02610",
                 "PMN2A_RS03005",
                 "PMN2A_RS07470",
                 "PMN2A_RS09305",
                 "PMN2A_RS09820"]

genes = annotation_data[annotation_data["locus_tag"].isin(sigma_factors)]

out_path = plotting_dir / 'sigma_factors.jpg'
plot_gene_table(genes, out_path, rlog_mean_df, rlog_std_df, result_df, annotation_data)


In [12]:
# Plot Sigma Factors
sigma_factors = ["PMN2A_RS02610",
                 "PMN2A_RS03005",
                 "PMN2A_RS07470",
                 "PMN2A_RS09305",
                 "PMN2A_RS09820"]

genes = annotation_data[annotation_data["locus_tag"].isin(sigma_factors)]

out_path = plotting_dir / 'sigma_factors.jpg'
plot_gene_table(genes, out_path, rlog_mean_df, rlog_std_df, result_df, annotation_data)
